In [1]:
'''
Reconstruction helical data using parallel conebeam rebinning
'''

'\nReconstruction helical data using parallel conebeam rebinning\n'

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

import ct_projector.projector.numpy as ct_projector
import ct_projector.projector.numpy.helical_equiangular_parallel_rebin as ct_helical

In [3]:
input_filename = '54_1.mat'

with h5py.File(input_filename, 'r') as f:
    # if a view is valid
    view_valid_a = np.copy(f['sh']['Lookup']['DetA']).flatten()
    view_valid_b = np.copy(f['sh']['Lookup']['DetB']).flatten()
    
    # z position of the source for each view, convert to mm
    zpos_a = np.copy(f['posA']).flatten() / 1000
    zpos_b = np.copy(f['posB']).flatten() / 1000
    
    # angle of the source for each view, convert to radius
    angles_a = np.copy(f['angleA']).flatten() / 180 * np.pi
    angles_b = np.copy(f['angleB']).flatten() / 180 * np.pi
    
    # projection, convert to attenuation
    prjs_a = np.copy(np.copy(f['projA'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5
    prjs_b = np.copy(np.copy(f['projB'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5

In [4]:
zpos = np.copy(zpos_a)
angles = np.copy(angles_a)
prjs = np.copy(prjs_a)

projector = ct_projector.ct_projector()
projector.nu = prjs.shape[2]
projector.nv = prjs.shape[1]
projector.dso = 595
projector.du = 0.067864004196156 * np.pi / 180 * projector.dsd
projector.dv = 1.0947
projector.off_u = -1.25
projector.rotview = 1152
dtheta = np.pi * 2 / projector.rotview

angles, zrot = ct_helical.convert_angles_and_calculate_pitch(angles, zpos)

In [5]:
projector_rebin, nview_margin_pre, nview_margin_post = ct_helical.get_rebin_geometry(projector)

In [7]:
prjs_rebin = ct_helical.rebin_to_parallel_conebeam(
    prjs,
    prjs.shape[0] - nview_margin_pre - nview_margin_post,
    projector_rebin.nu,
    (projector_rebin.nu - 1) / 2 + projector_rebin.off_u,
    projector_rebin.du,
    (projector.nu - 1) / 2 + projector.off_u,
    0.067864004196156 * np.pi / 180,
    projector.dso,
    dtheta,
    angles[nview_margin_pre],
    angles[0]
)

Beta (fan angle) interpolation...
100,200,300,400,500,600,700,
Theta (source angle) interpolation...
100,200,300,400,500,600,700,
